This guide start programming in the low-level TensorFlow APIs (TensorFlow Core), showing:
- Manage own TensorFlow program (a `tf.Graph`) and TensorFlow runtime (a `tf.Session`), instead of relying on Estimators to manage them.
- Run TensorFlow operations, using a `tf.Session`.
- Using high level components (`dataset`, `layers`, and `feature_column`) in this low level environment.
- Build own training loop, instead of using the one `provided by Estimators`.

Recommend using the higher level APIs to build models when possible. Know TensorFlow Core is valuable for the folowwing reasons:
- Experimentation and debugging are both more straight forward when you can use low level TensorFlow operations directly.
- It gives you a mental model of how things work internally when using the higher level APIs.

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

### Tensor values
The central unit of data in TensorFlow is the **tensor**. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's **rank** is its number of dimension, while its **shape** is a tuple of integers specifying the array's length along each dimension.

In [0]:
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

TensorFlow uses numpy array to represent tensor **values**.

### TensorFlow Core Walkthrough
You might think of TensorFlow Core programs as consisting of two discrete sections:
1. Building the computational graph (a `tf.Graph`).
2. Running the computational graph (using a `tf.Session`).


### Graph
A **computational graph** is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.
- `tf.Operation` (or "obs"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
- `tf.Tensor`: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return `tf.Tensor`.

**Important**: tf.Tensor do not have values, they are just handles to elements in the computation graph.

Let's build a simple computational graph. The most basic operation is a constant. The Python function that builds the operation takes a tensor value as input. The resulting operation takes no input. When run, it outputs the value that was passed to the constructor.

In [0]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float 32 implicitly
total = a + b
print(a)
print(b)
print(total)

Notice that printing the tensors does not output the values `3.0`, `4.0`, and `7.0` as you might expect. The above statements only build the computational graph. These `tf.Tensor` objects just represent the results of the operations that will be run.

Each operation in a graph is given a unique name. This name is independent of the names the objects are assigned to in Python. Tensors are named after the operation that produces them followed by an output index.

### TensorBoard
TensorFlow provides a utility called TensorBoard. One of TensorBoard's many capabilities is visualizing a computation graph. You can easily do this with a few simple commands.

First you save the computation graph to a TensorBoard summary file as follows:

In [0]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

This will produce an `event` file in the current directory with a name in the following format:

In [0]:
events.out.tfevents.{timestamp}.{hostname}
tensorboard --logdir .

### Session
To evaluate tensors, instantiate a `tf.Session` object, informally known as **session**. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations. If a `tf.Graph` is like `.py` file, a `tf.Session` is like the Python executable.

The following code create a `tf.Session` object and then invokes its `run` method to evaluate the `total` tensor we create above:

In [0]:
sess = tf.Session()
print(sess.run(total))

When you request toe output of a node with `Session.run` TensorFlow backtracks through graph and runs all the nodes that provide input to the requested output node.

You can pass multiple tensors to `tf.Session.run`. The `run` method transparently handles any combination of tuples or dictionaries

In [0]:
print(sess.run({'ab': (a, b), 'total': total}))

During a call to `tf.Session.run` any `tf.Tensor` only has a single value. For example, the following code call `tf.random_uniform` to produce a `tf.Tensor` that generates a random 3-element vector (with values in [0, 1)):

In [0]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

Some TensorFlow functions return `tf.Operation` instead of `tf.Tensor`. The result of calling `run` on an Operation is `None`. You run an operation to cause side-effect, not to retrieve a value. Examples of this include the `initialization`, and `training` ops demonstrated later.

### Feeding
As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as **placeholder**. A placeholder is a promise to provide a value later, like a function argument.

In [0]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [0]:
print(x)
print(y)
print(z)

The preceding three lines are a bit like a function in which we define two input parameters (x and y) and then an operation on them. We can evaluate this graph with multiple inputs by using the feed_dict argument of the tf.Session.run method to feed concrete values to the placeholders:

In [0]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

Also note that the `feed_dict` argument can be used to overwrite any tensor in the graph. The `only` difference between placeholders and other `tf.Tensor` is that placeholders throw an error if no value is feed to them.

### Datasets
Placeholders work for simple experiments, but `tf.data` are the preferred method of streaming data into a model.

To get a runnable `tf.Tensor` from a Dataset you must first convert it to a `tf.data.Iterator`, and then call the Iterator's `tf.data.Iterator.get_next` method.

The simplest way to create an Iterator is with the `tf.data.Dataset.make_one_shot_iterator` method. For example, in the following code the `next_item` tensor will return a row from the `my_data` array on each run call:

In [0]:
my_data = [
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7]
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

Reaching the end of the data stream causes `Dataset` to throw an `tf.errors.OutOfRangeError`.

In [0]:
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

If the `Dataset` depends on stateful operations you may need to initialize the iterator before using it.

In [0]:
r = tf.random_normal([10, 3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

### Layers
A trainable model must modify the values in the graph to get new outputs with the same input. `tf.layers` are the preferred way to add trainable parameters to a graph.

Layers package together both the variables and the operations that act on them. For example `densely-connected layer` performs a weighted sum across all inputs for each output and applies an optional `activation function`. The connection weights and biases are managed by the layer object.

### Creating layer

In [0]:
# Takes a batch of input vectors, and produce single value for each.
# To apply a layer to an input, call the layer as if it were a function
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

The layer inspects its input to determine sizes for its internal variable. So here we must set the shape of the `x` placeholder so that the layer can build a weight matrix of the correct size.

Now that we have defined the calculation of the output, y, there is one more detail we need to take care of before we run the calculation.

In [0]:
sess.run(y, feed_dict={x: [[1, 2, 3]]})

### Initializing Layers
The layer contains variables that must be **initialized** before they can be used. While it is possible to initialize variables individually, hou can easily initialize all the variables in a TensorFlow graph as follows:

In [0]:
init = tf.global_variables_initializer()
sess.run(init)

In [0]:
print(sess.run(y, {x: [[1, 2, 3]]}))

### Layer Function shortcuts
For each layer class (like tf.layers.Dense) TensorFlow also supplies a shortcut function (like tf.layers.dense). The only difference is that the shortcut function versions create and run the layer in a single call. For example, the following code is equivalent to the earlier version:

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

While convenient, this approach allows no access to the tf.layers.Layer object. This makes introspection and debugging more difficult, and layer reuse impossible.

### Feature columns


In [0]:
features = {
    'sales': [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
    'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)


Feature columns can have internal state, like layers, so they often need to be initialized. Categorical columns use tf.contrib.lookup internally and these require a separate initialization op, tf.tables_initializer.

In [0]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

In [0]:
print(sess.run(inputs))

In [0]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

In [0]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

In [0]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

In [0]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [0]:
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

In [0]:
x = tf.constant([[1], [2], [3], [4]],dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
  _ , loss_value = sess.run((train, loss))
  print(loss_value)
print(sess.run(y_pred))